### Trying different approaches for prediction

In [2]:
import pandas as pd

def read_parquet_file(file_path):
    """
    Reads a Parquet file and returns a DataFrame.
    
    :param file_path: Path to the Parquet file.
    :return: DataFrame containing the data from the Parquet file.
    """
    try:
        df = pd.read_parquet(file_path, engine='pyarrow')
        return df
    except Exception as e:
        print(f"Error reading Parquet file: {e}")
        return None

df = read_parquet_file('../data/processed/ratings_movies.parquet')

In [3]:
df.head()

,userId,movieId,rating,genre_list,(no genres listed),Action,Adventure,Animation,Children,Comedy,...,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,17,4.0,"[Drama, Romance]",0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
1,1,25,1.0,"[Drama, Romance]",0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,1,29,2.0,"[Adventure, Drama, Fantasy, Mystery, Sci-Fi]",0,0,1,0,0,0,...,0,0,0,0,1,0,1,0,0,0
3,1,30,5.0,"[Crime, Drama]",0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,32,5.0,"[Mystery, Sci-Fi, Thriller]",0,0,0,0,0,0,...,0,0,0,0,1,0,1,1,0,0
